In [19]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from sklearn.metrics import accuracy_score, r2_score
from models import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:

df = pd.read_csv(datafile)
df["date"] = pd.to_datetime(df["day_survey"])
df['C'] = df['date'].apply(lambda date: COVIDStatus.POST_COVID if date > date_covid else COVIDStatus.PRE_COVID)

df_head = df.head(5).copy()

df.rename(columns=reverse_ema_dictionary, inplace=True)
df.set_index(["uid", "date"], inplace=True)
df.dropna(subset=ema + physical + social + sleep, inplace=True)

sets_df = pd.read_parquet(sets_file, engine='pyarrow')

In [21]:
index = 51
model_rows = [sets_df.iloc[index]]
#for index, model_row in sets_df.iterrows():

alphas = [0.1, 0.2]
gammas = [0.3, 0.5]

for alpha in alphas:
    for gamma in gammas:
        for model_row in model_rows:
            print(f"Modeling yreatment:{full_dictionary[model_row['treatment']]} on outcome:{full_dictionary[model_row['outcome']]}")
            # people can't change their demographics.. except maybe OS :)
            if model_row['treatment'].startswith('D'):
                print(f"Skippnig demographic treatment {model_row['treatment']}")
                continue
            wbm = WBKernelModel(
                data=df,
                gamma=gamma,
                alpha=alpha,
                #activation='relu',
                #max_iter=1000,
                #hidden_layer_sizes=(32,8,4),
                treatment=model_row['treatment'],
                outcome=model_row['outcome'],
                separating_set=model_row['sets'].tolist(),
                name=f"row:{index}"
                )

            print(f"gamma={gamma}, alpha={alpha}, pre_rsq={wbm.pre_r_squared} post_r_sq={wbm.post_r_squared}")

Modeling yreatment:duration unlocked phone in social locations on outcome:phq2_score
Binarizing using median value
Directory 20240527_201854_734872 created.
gamma=0.2, alpha=0.1, pre_rsq=(0.6266832563645484, 0.23019788131897034) post_r_sq=(0.7644432161302293, 0.185420484652273)
Modeling yreatment:duration unlocked phone in social locations on outcome:phq2_score
Binarizing using median value
Directory 20240527_201909_363658 created.
gamma=0.3, alpha=0.1, pre_rsq=(0.741561353799443, 0.20685635832817573) post_r_sq=(0.8412399071948569, 0.21219028093323777)
Modeling yreatment:duration unlocked phone in social locations on outcome:phq2_score
Binarizing using median value
Directory 20240527_202338_459825 created.
gamma=0.5, alpha=0.1, pre_rsq=(0.8531888225168781, 0.20949923086481392) post_r_sq=(0.9139538618253462, 0.1678845330994968)
Modeling yreatment:duration unlocked phone in social locations on outcome:phq2_score
Binarizing using median value
Directory 20240527_202354_184046 created.
gamm

In [22]:
results_df = aggregate_results_from_subfolders('.')
results_df.to_csv('results.csv')